In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import re


In [2]:
def get_product_name(product_name):
    """Làm sạch tên sản phẩm, bỏ phần trong ngoặc."""
    return re.sub(r"\(.*\)", "", product_name).strip()


In [10]:
def get_all_product_links(driver, url):
    driver.get(url)
    time.sleep(2)

    while True:
        try:
            button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'a.css-b0m1yo'))
            )

            # Cuộn đến nút và dùng JS để click
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(0.5)

            # Dùng JS click để tránh bị intercept
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)

        except TimeoutException:
            break
        except Exception as e:
            print(f"Gặp lỗi khi click: {e}")
            break

    # Lấy tất cả link sản phẩm
    product_links = []
    product_elements = driver.find_elements(By.CSS_SELECTOR, '.product-card a.css-pxdb0j')
    for element in product_elements:
        href = element.get_attribute("href")
        if href:
            full_link = "https://phongvu.vn" + href if href.startswith("/") else href
            product_links.append(full_link)

    print(f"Đã lấy tổng cộng {len(product_links)} link sản phẩm.")
    return product_links


In [ ]:
def crawl_page(driver):
    crawl_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    print(f"Crawl time: {crawl_time}")
    specs = {}

    # Lấy tên sản phẩm
    try:
        name_elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.css-nlaxuc'))
        )
        specs['Tên sản phẩm'] = get_product_name(name_elem.text.strip())
    except Exception as e:
        print(f"Không lấy được tên sản phẩm: {e}")

    # Lấy giá bán
    try:
        price_elem = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.att-product-detail-latest-price'))
        )
        specs['Giá bán'] = price_elem.text.strip()
    except Exception as e:
        print(f"Không lấy được giá bán: {e}")

    try:
        found = False
        for _ in range(10):
            driver.execute_script("window.scrollBy(0, 500);") 
            time.sleep(1)  

            try:
                btn = driver.find_element(By.CSS_SELECTOR, "div.css-1alns4t")
                if btn.is_displayed():
                    driver.execute_script("arguments[0].click();", btn)
                    print("Đã bấm 'Xem thêm nội dung'")
                    found = True
                    break
            except:
                pass  

        if not found:
            print("Không tìm thấy nút 'Xem thêm nội dung' sau khi cuộn")

    except Exception as e:
        print(f"Lỗi khi xử lý nút: {e}")

    # Lấy thông số kỹ thuật
    try:
        container = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.css-1ilwpbr'))
        )
        items = container.find_elements(By.CSS_SELECTOR, 'div.css-1i3ajxp')
        for item in items:
            try:
                key_val = item.find_elements(By.CSS_SELECTOR, 'div.css-1lchwqw')
                if len(key_val) == 2:
                    key = key_val[0].text.strip()
                    value = key_val[1].text.strip()
                    specs[key] = value
            except:
                continue
    except Exception as e:
        print(f"Lỗi khi lấy thông số kỹ thuật: {e}")

    return specs


In [11]:
import pandas as pd
import os

def save_to_csv(data_list, filename="../crawl_data/phongvu.csv"):
    if not data_list:
        print("Không có dữ liệu để lưu.")
        return

    df = pd.DataFrame(data_list)

    # Ghi đè file CSV mỗi lần
    df.to_csv(filename, mode='w', index=False, encoding="utf-8", header=True)

    print(f"Đã lưu {len(data_list)} sản phẩm vào '{filename}'.")


In [12]:
all_data = []
url = "https://phongvu.vn/c/laptop"  
options = webdriver.ChromeOptions()
options.headless = False  
driver = webdriver.Chrome(options=options)

try:
    links = get_all_product_links(driver, url)  
    print(f"Đã lấy {len(links)} link sản phẩm.")

    for i, link in enumerate(links, start=1):
        print(f"Đang xử lý link {i}/{len(links)}: {link}")
        try:
            driver.get(link)
            time.sleep(2)  # Chờ trang tải

            data = crawl_page(driver)
            all_data.append(data)
            print(f"✅ Đã lấy dữ liệu từ {link}")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý {link}: {e}")
            continue

finally:
    driver.quit()

save_to_csv(all_data, "../raw_data/phongvu.csv")

Đã lấy tổng cộng 833 link sản phẩm.
Đã lấy 833 link sản phẩm.
Đang xử lý link 1/833: https://phongvu.vn/laptop-msi-titan-18-hx-ai-a2xwjg-035vn--s250116531
Crawl time: 2025-05-24 04:11:38
Đã bấm 'Xem thêm nội dung'
✅ Đã lấy dữ liệu từ https://phongvu.vn/laptop-msi-titan-18-hx-ai-a2xwjg-035vn--s250116531
Đang xử lý link 2/833: https://phongvu.vn/laptop-msi-titan-18-hx-ai-a2xwig-090vn--s250116532
Crawl time: 2025-05-24 04:11:51
Đã bấm 'Xem thêm nội dung'
✅ Đã lấy dữ liệu từ https://phongvu.vn/laptop-msi-titan-18-hx-ai-a2xwig-090vn--s250116532
Đang xử lý link 3/833: https://phongvu.vn/laptop-acer-nitro-v-15-propanel-anv15-51-56d5--s250203153
Crawl time: 2025-05-24 04:12:04
Đã bấm 'Xem thêm nội dung'
✅ Đã lấy dữ liệu từ https://phongvu.vn/laptop-acer-nitro-v-15-propanel-anv15-51-56d5--s250203153
Đang xử lý link 4/833: https://phongvu.vn/laptop-acer-nitro-lite-16-nl16-71g-56wq--s250409681
Crawl time: 2025-05-24 04:12:18
Đã bấm 'Xem thêm nội dung'
✅ Đã lấy dữ liệu từ https://phongvu.vn/laptop

KeyboardInterrupt: 